# Imports & Settings


In [1]:
from setup_general import *

In [2]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})
test_prep = pd.read_csv('data/test_prepared.csv', index_col='id', dtype={'type': str})
train_prep = pd.read_csv('data/train_prepared.csv', index_col='id', dtype={'type': str})

smote100_full = pd.read_csv('data/train_prepSMOTE100.csv', dtype={'type': str})

# xg boost


In [12]:
data = high.copy()
data = data[data.source=='train']
data = data.drop(columns=['source'])

In [4]:
data = smote100_full.copy()
data.drop(columns=['Unnamed: 0'], inplace=True, axis = 1)

: 

In [11]:
features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

In [12]:
bst = XGBClassifier(random_state=0)
# fit model
bst.fit(features, labels)
# make predictions
#preds = bst.predict(X_test)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [19]:
val_acc = accuracy_score(y_test, preds)
val_acc
#0.9002380952380953
#0.9007142857142857

0.9007142857142857

In [13]:
bst.save_model('models/xg/full_smote100.json')

# submission from model

In [86]:
test_set = test_prep.drop('type', axis=1)
results = bst.predict(test_set)
submission = pd.DataFrame({'id': test_set.index ,'type': bst.predict(test_set)})
type_lookup = pd.read_csv('data/type_lookup.csv')
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
submission.to_csv('data/submission_xgboost_full.csv', index=False)

# class name mapping

In [29]:
estonian = train.type.unique()

In [30]:
english = train_prep.type.unique()

In [48]:
type_lookup = pd.DataFrame({'estonian': estonian, 'english': english})

In [49]:
type_lookup = type_lookup.sort_values(by='english')

In [50]:
type_lookup.reset_index(drop=True, inplace=True)
type_lookup['id'] = type_lookup.index

In [52]:
type_lookup.to_csv('data/type_lookup.csv')